In [2]:
from modules.facebook_operate import FacebookOperate

browser_id = '6875eea406de409a8449b16e2e79a54d'
fb_operate = FacebookOperate(browser_id,capture=True)
print(fb_operate.browser_control.browser_data)

C:\Users\lqj\AppData\Roaming\BitBrowser\chromedriver\126\chromedriver.exe
127.0.0.1:64754
{'id': '6875eea406de409a8449b16e2e79a54d', 'seq': 78, 'code': '20240723150758833', 'groupId': '2c9bc04790aae7ca0190b5cb34182ea7', 'platform': '', 'platformIcon': '', 'url': 'https%3A%2F%2Fwww.google.com%2F,https%3A%2F%2Fwww.facebook.com%2Fmessages%2Ft', 'name': '61563533240642-收1消息', 'userName': '', 'password': '', 'cookie': '[{"name":"sb","value":"2lafZsgD4n0g3eqR9JQRfoBt","domain":".facebook.com","path":"/","expires":1753515742,"httpOnly":true,"secure":true,"session":false,"sameSite":"None","sameParty":false},{"name":"datr","value":"2lafZlx2M5XyXcrt1Z_zON4U","domain":".facebook.com","path":"/","expires":1753515742,"httpOnly":true,"secure":true,"session":false,"sameSite":"None","sameParty":false},{"name":"locale","value":"zh_HK","domain":".facebook.com","path":"/","expires":1753515742,"httpOnly":false,"secure":true,"session":false,"sameSite":"None","sameParty":false},{"name":"c_user","value":"615

In [ ]:
import json
browser_data = fb_operate.browser_control.browser_data
print(json.dumps(browser_data))

proxy_host = browser_data.get('host')
proxy_port = browser_data.get('port')
proxy_user = browser_data.get('proxyUserName')
proxy_pass = browser_data.get('proxyPassword')

proxy_url = f'socks5://{proxy_user}:{proxy_pass}@{proxy_host}:{proxy_port}'

wire_options = {
    'proxy': {
        'http':  proxy_url,
        'https': proxy_url,
        'no_proxy': 'localhost,127.0.0.1'
    }
}

In [63]:
# from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

# ca = DesiredCapabilities.CHROME
# ca["goog:loggingPrefs"] = {"performance": "ALL"}
# driver = fb_operate.driver(desired_capabilities=ca)
import json
# print(logs)
def process_network_event(driver, logs, match_url):
    for entry in logs:
        message = json.loads(entry["message"]).get("message", {})
        method = message.get("method", "")
        
        url = message.get("params", {}).get("response", {}).get("url", "")
        # print(url)
        if url == "":
            continue
        if not url.startswith(match_url): # 匹配我们想要的url
            continue
        print(method)
        if method == "Network.requestWillBeSent":
            if message['params']['request']['url'].startswith(match_url):
                request_id = message['params']['requestId']
                request_data = message['params']['request']['postData'] if 'postData' in message['params']['request'] else None

                # 在这里，你可以处理请求数据
                print(f"Request Data for {request_id}: {request_data}")

        elif method == "Network.responseReceived":
            response = message['params']['response']
            if response['url'].startswith(match_url):
                request_id = message['params']['requestId']

                # 获取响应体
                try:
                    response_body = driver.execute_cdp_cmd('Network.getResponseBody', {'requestId': request_id})
                    json_string = response_body.get("body", "")
                    if json_string:
                        response_data = json.loads(json_string)
                        # 在这里，你可以处理响应数据
                        print(f"Response Data for {request_id}: {response_data}")
                except Exception as e:
                    print(f"Error getting response body for {request_id}: {e}")

    return None  # 或者根据你的需要返回适当的数据

In [57]:

logs = fb_operate.driver.get_log("performance")
print(len(logs))

857


In [64]:
print(len(logs))
print(len(logs))
res = process_network_event(fb_operate.driver,logs,'https://www.facebook.com/api/graphql/')
print(res)

857
857
Network.responseReceived
Response Data for 39800.516: {'data': {'feedback': {'reaction_display_config': {'reaction_display_strategy': 'NONE'}, 'important_reactors': {'nodes': []}, 'reactors': {'nodes': [{'__typename': 'User', 'name': 'Wen Tseng', 'id': '100000080411233'}, {'__typename': 'User', 'name': '小柴摩摩', 'id': '100032210144762'}, {'__typename': 'User', 'name': '李芸茹', 'id': '100001997827288'}, {'__typename': 'User', 'name': '張觀貞', 'id': '100035627623950'}, {'__typename': 'User', 'name': '陳寶惠', 'id': '100009212135813'}, {'__typename': 'User', 'name': '陳郁秋', 'id': '100008650810716'}, {'__typename': 'User', 'name': 'Susi Hsieh', 'id': '100024660140605'}, {'__typename': 'User', 'name': '謝美珠', 'id': '100006142818879'}, {'__typename': 'User', 'name': '賴玉嬌', 'id': '100008524115191'}, {'__typename': 'User', 'name': 'Pauline Wong', 'id': '100063927406549'}, {'__typename': 'User', 'name': '徐純惠', 'id': '100008351895858'}, {'__typename': 'User', 'name': '何貔貅', 'id': '100026182863109'}

In [48]:
fb_operate.driver.get('https://www.facebook.com/photo/?fbid=1315274582889534&set=pcb.1315274616222864&__cft__[0]=AZW04tnp1oX0PbuOR1_RHPM1BoJzuh615Gv8ycgFQrUXoHOfJFEh5gn1fnTwoEpqPe6TJE4rn1RQlvA7BSb8EO9dvsaqLAoT5bboJL9DWrtyFtDBH2QWM6Gz7oi45HdmsLTWWFmjciJ9EfKVjl2sJBUo6BSTl5avrQ2M61fIKzkcuK-xxMQTytzDGv0WpJ9SyH-R_jWh1BOFUaT7cFji7xRrHyCEzEw5D8PNUmG0pdkdaw&__tn__=*bH-R')
# req = fb_operate.driver.requests()
# print(req)

In [ ]:
print(fb_operate.driver.requests())

In [ ]:
fb_id = fb_operate.fetch_facebook_id()
print(fb_id)

In [ ]:
fb_operate.driver.get('https://www.facebook.com')
print(fb_operate.driver.requests)
for request in fb_operate.driver.requests:
    print("-----")
    if request.response:
        print(
            request.url,
            request.response.status_code,
            request.response.headers['Content-Type'],
            request.body
        )

In [ ]:
from seleniumwire import webdriver  # Import from seleniumwire

# Create a new instance of the Chrome driver
driver = webdriver.Chrome()

# Go to the Google home page
driver.get('https://www.google.com')

# Access requests via the `requests` attribute
print(driver.requests)
for request in driver.requests:
    if request.response:
        print(
            request.url,
            request.response.status_code,
            request.response.headers['Content-Type'],
            request.body
        )